In [ ]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
from sklearn.svm import SVC
from sklearn import metrics
from sklearn.metrics import confusion_matrix, classification_report

class SVM:
    def __init__(self,feature_matrix_train,y_train):
        self.feature_matrix_train = feature_matrix_train
        self.y_train = feature_matrix_train
        self.clf = SVC(kernel='linear')
        self.clf.fit(feature_matrix_train, y_train)

    def predictSvm(self,feature_matrix_test,y_test):
        clf_predictions = self.clf.predict(feature_matrix_test)
        test_accuracy = str(metrics.accuracy_score(y_test, clf_predictions))
        print (confusion_matrix(y_test, clf_predictions))
        print ('\n')
        print (classification_report(y_test, clf_predictions))

        return test_accuracy

    def predictRating(self,X_test):
        clf_predictions = self.clf.predict(X_test)
        return clf_predictions

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
from sklearn.metrics import confusion_matrix, classification_report

class RF:
    def __init__(self, feature_matrix_train, y_train):
        self.feature_matrix_train = feature_matrix_train
        self.y_train = feature_matrix_train
        self.clf = RandomForestClassifier()
        self.clf.fit(feature_matrix_train, y_train)

    def predictRF(self,feature_matrix_test,y_test):
        clf_predictions = self.clf.predict(feature_matrix_test)
        test_accuracy = str(metrics.accuracy_score(y_test, clf_predictions))
        print (confusion_matrix(y_test, clf_predictions))
        print ('\n')
        print (classification_report(y_test, clf_predictions))

        return test_accuracy

    def predictRating(self,X_test):
        clf_predictions = self.clf.predict(X_test)
        return clf_predictions


In [ ]:
import pandas as pd
from nltk.corpus import stopwords
import sklearn as sk
import re
#from svm import SVM
#from NaiveBayes import NB
from sklearn.model_selection import train_test_split
import seaborn as sns
import matplotlib.pyplot as plt
#from rf import RF
import random

from google.colab import drive
drive.mount("/content/drive")
reviews = pd.read_csv("/content/drive/My Drive/data/review.csv", nrows = 10000)

w_reviews = reviews[(reviews['stars'] == 1) | (reviews['stars'] == 5) | (reviews['stars'] == 3)]
goodWords = [] # list of positive words
badWords = [] # list of
reviews_text = w_reviews['text'] # text of the reviews on which sentiment analysis will be performed
reviews_star = w_reviews['stars'] # label of the review


def remove_punc(text):
	letters_only = re.sub('[^a-zA-Z]', ' ',text)
	words = letters_only.lower().split()
	return remove_stopwords(words)

def remove_stopwords(words):
	stopwords_eng = set(stopwords.words("english"))
	useful_words = [word for word in words if not word in stopwords_eng]
	return(useful_words)

# feature vector matrix
feature_matrix = sk.feature_extraction.text.CountVectorizer(analyzer = remove_punc).fit(reviews_text)
reviews_text_transformed = feature_matrix.transform(reviews_text)

# spliting of the feature vector in training and testing
X_train, X_test, y_train, y_test = train_test_split(reviews_text_transformed, reviews_star, test_size=0.3, random_state=101)


# Data analysis
reviews['text length'] = reviews['text'].apply(len)
def graph():
    g = sns.FacetGrid(data=reviews, col='stars')
    g.map(plt.hist, 'text length', bins=50)
    plt.show()
    sns.boxplot(x='stars', y='text length', data=reviews)
    plt.show()


#naive bayes algorithm
def naiveBayesTrain():
    bayesian_reviews_train = reviews[0:9000] # training set
    five_star_reviews = bayesian_reviews_train['text'][(bayesian_reviews_train['stars'] == 5)] # getting text of five star reviews
    one_star_reviews = bayesian_reviews_train['text'][(bayesian_reviews_train['stars'] == 1)] # getting text of one star reviews

    for text in five_star_reviews:
        for word in remove_punc(text):
            goodWords.append(word) # creating good words list

    for text in one_star_reviews:
        for word in remove_punc(text):
            badWords.append(word) # creating bad words list


def naiveBayesPredict(text):
    review_words = remove_punc(text) # clean review
    goodWordsCounter = 0
    badWordsCounter = 0
    length_of_review = len(review_words)
    for word in review_words:
        if word in goodWords:
            goodWordsCounter +=1
        if word in badWords:
            badWordsCounter +=1

    # calculate probability
    if length_of_review > 0:
        goodProb = float(goodWordsCounter) / length_of_review #conditional property of review being good
        badProb = float(badWordsCounter) / length_of_review # probabilty of review being bad

        if goodProb < badProb:
            return 1
        elif goodProb > badProb:
            return 5
        else:
            flip = random.randint(0,1)
            if flip == 0:
                return 1
            else:
                return 5


    else:
        return 0
def svm():
    print ("SVM model:")
    svm = SVM(X_train,y_train)
    test_accuracy = svm.predictSvm(X_test,y_test)
    our_review = "The food was good but the drinks was bad"
    print ("our test for sample text :: ",our_review)
    our_review_transformed = feature_matrix.transform([our_review])
    print ("Rating of our review",svm.predictRating(our_review_transformed))

def Nb():

    print("naive bayes model:")
    naiveBayesTrain()
    bayesian_reviews_test = reviews[6000:10000]
    five_star_reviews = bayesian_reviews_test['text'][(bayesian_reviews_test['stars'] == 5)]
    one_star_reviews = bayesian_reviews_test['text'][(bayesian_reviews_test['stars'] == 1)]
    five_star_correct_prediction = 0
    one_star_correct_prediction = 0
    pos_review = w_reviews['text'][0]
    neg_review = w_reviews['text'][16]
    for review in five_star_reviews:
        if naiveBayesPredict(review) == 5:
            five_star_correct_prediction += 1
    for review in one_star_reviews:
        if naiveBayesPredict(review) == 1:
            one_star_correct_prediction +=1

    five_correctly_predicted = float(five_star_correct_prediction) / len(five_star_reviews)
    one_correctly_predicted = float(one_star_correct_prediction) / len(one_star_reviews)
    print ("efficiency of predicting positive reviews ", 100 * five_correctly_predicted)
    print ("efficiency of predicting negative reviews ", 100 * one_correctly_predicted)



print ("Calculated probability of a review being a positive or negative one")
Nb()
svm()
